In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/maternal-and-child-health-monitoring-in-lmics/training_label.csv
/kaggle/input/maternal-and-child-health-monitoring-in-lmics/sample submission.csv
/kaggle/input/maternal-and-child-health-monitoring-in-lmics/gee_features.csv
/kaggle/input/selecting-features-using-tree/y_train.csv
/kaggle/input/selecting-features-using-tree/x_train.csv
/kaggle/input/selecting-features-using-tree/__results__.html
/kaggle/input/selecting-features-using-tree/x_submission.csv
/kaggle/input/selecting-features-using-tree/__notebook__.ipynb
/kaggle/input/selecting-features-using-tree/DHSID.csv
/kaggle/input/selecting-features-using-tree/__output__.json
/kaggle/input/selecting-features-using-tree/custom.css


In [2]:
from scipy.stats import pearsonr

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
x_train = pd.read_csv('/kaggle/input/selecting-features-using-tree/x_train.csv')
y_train = pd.read_csv('/kaggle/input/selecting-features-using-tree/y_train.csv')
x_submission = pd.read_csv('/kaggle/input/selecting-features-using-tree/x_submission.csv')
DHSID = pd.read_csv('/kaggle/input/selecting-features-using-tree/DHSID.csv')

In [4]:
from sklearn.model_selection import train_test_split

# Set the random seed for reproducibility
random_seed = 42
np.random.seed(random_seed)

# Splitting the data into train and test sets based on country (80% train, 20% test)
x_train1, x_test1, y_train1, y_test1 = train_test_split(x_train, y_train, test_size=0.2, stratify = x_train["key1"])

# Splitting the train set into train and val sets based on country (75% train, 25% dev)
x_train1, x_val1, y_train1, y_val1 = train_test_split(x_train1, y_train1, test_size=0.25, stratify = x_train1["key1"])

In [5]:
column_names = y_train.columns

In [6]:
import xgboost as xgb

In [ ]:
# Evaluating the model on the testing and validation sets
for column in column_names:
    # Training an XGBoost model for each target column
    model = xgb.XGBRegressor()
    model.fit(x_train1, y_train1[column])

    # Evaluating the model on the testing set
    y_test_pred = model.predict(x_test1)
    r_test, _ = pearsonr(y_test1[column], y_test_pred)
    pearson_r2_test = r_test ** 2
    print("Squared Pearson r^2 of x_test column {}: {}".format(column, pearson_r2_test))

    # Evaluating the model on the validation set
    y_val_pred = model.predict(x_val1)
    r_val, _ = pearsonr(y_val1[column], y_val_pred)
    pearson_r2_val = r_val ** 2
    print("Squared Pearson r^2 of x_val column {}: {}".format(column, pearson_r2_val))

In [8]:
y_pred = {}
for column in column_names:
    # Training XGBoost model for each target column
    model = xgb.XGBRegressor()
    model.fit(x_train, y_train[column])

    # Evaluate the model on the testing set
    y_pred[column] = model.predict(x_submission)
    print('Prediction of column {} completed'.format(column))

Prediction of column Mean_BMI completed
Prediction of column Median_BMI completed
Prediction of column Unmet_Need_Rate completed
Prediction of column Under5_Mortality_Rate completed
Prediction of column Skilled_Birth_Attendant_Rate completed
Prediction of column Stunted_Rate completed


In [9]:
submission_df2 = pd.DataFrame(y_pred)

In [10]:
# Inserting the 'DHSID' column as the first column in the DataFrame
submission_df2.insert(0, 'DHSID', DHSID)

In [11]:
submission_df2

,DHSID,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
0,AL200800000003,25.094822,24.330502,23.142097,1.205332,73.498665,10.881701
1,AL200800000005,25.040625,24.391668,28.218515,2.749399,77.806351,12.366354
2,AL200800000007,25.021967,24.070175,22.175745,2.990956,79.531364,7.885830
3,AL200800000008,25.223866,24.844107,21.978218,2.618988,67.758560,6.826309
4,AL200800000009,24.982500,24.566111,17.943075,2.391696,74.241348,6.887013
...,...,...,...,...,...,...,...
14995,ZW201500000382,23.604536,22.934731,17.954775,5.952562,77.221573,32.092106
14996,ZW201500000383,23.430618,23.524963,10.775187,5.189237,81.235451,24.008869
14997,ZW201500000386,24.428640,23.329235,15.427325,4.852790,89.793739,24.801233
14998,ZW201500000390,24.731043,24.165565,11.612862,5.981121,85.437630,24.927443


In [12]:
submission_df2.to_csv('submission2.csv',index = False)